In [14]:
import re
import os
import sys
import pickle
import time
from datetime import datetime
import logging

import scanpy as sc
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE,VarianceThreshold,SelectKBest,chi2, SelectFromModel, f_classif, mutual_info_classif
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt

In [6]:
def convert_data(data = None, path = None, assay = ".X",label_column = None):
    """
    A function to convert sparse matrix into pandas data frame object. 
    
    Parameters:
    data: a Annot Data object. If speciefied, path should be specified in 'None'.
    path: path to h5ad file. If speciefied, data should be specified in 'None'.
    assay: ".X" or "raw" assay in Annot Data object could be specified
    label_column: the name of cell type column in the h5ad file. 
    If specified, the cell type column will be added into output.
    """
    if path != None:
        data = sc.read(path)
    
    if assay == ".X":
        counts = pd.DataFrame.sparse.from_spmatrix(data.X)
    else:
        counts = pd.DataFrame.sparse.from_spmatrix(data.raw.X)
        
    features = data.raw.var_names.tolist()
    index  = data.raw.obs_names.tolist()
    
    counts.columns = features
    counts.index = index
    
    if label_column != None:
        try:
            labels = data.obs[label_column].tolist()
            counts["cell_type"] = labels
        except:
            raise ValueError("The length of cell type column is not consistent with matrix")

    return counts

In [7]:
def quantile_normalize(data):
    """
    A function to do quantile normalization.  
    
    """
    data = data.loc[:, data.columns != "cell_type"]
    ranks = (data.rank(method = "first").stack())
    rank_mean = (data.stack().groupby(ranks).mean())
    # add interproblated values in between ranks
    finer_ranks = ((rank_mean.index + 0.5).tolist() + rank_mean.index.tolist())
    rank_mean = rank_mean.reindex(finer_ranks).sort_index().interpolate()
    data = data.rank(method = "average").stack().map(rank_mean).unstack()
    
    return data
    

In [8]:
def record_time():
    """
    A function to call out the current time.
    """
    current_time_stamp = time.time()
    date_time = datetime.fromtimestamp(current_time_stamp)
    return date_time.strftime("%Y-%m-%d %H:%M:%S")

In [10]:
def pre_processing(path, 
                   label_column,
                   assay = ".X", 
                   convert = True,
                   log_normalize = True, 
                   scale_data = False, 
                   quantile_normalize = False,
                   save = False):
    """
    A function to do preliminary normalization, encoding label and convert data into pandas data frame for downstream sklearn-based
    machine learning workflow.
    
    Parameters:
    path: path to the h5ad file.
    assay: ".X" or "raw" assay in Annot Data object could be specified.Default is ".X".
    label_column: The name of cell type column in the h5ad file.If specified, the cell type column will be added into output.
    convert: Bool value to decide whether convert Annot Data object into pandas data frame object.
    log_normalize: Bool value to decide whether standard log normalization to be done.
    scale_data: Bool value to decide whether standadlize data or not.
    quantile_normalize: Bool value to decide whether quantile normalize data or not.
    save: Bool value to decide whether write the pre-processed data into the disk.
    """
    data = sc.read(path)
    
    if log_normalize:
        sc.pp.normalize_total(data, target_sum = 1e4)
        sc.pp.log1p(data)
    
    if scale_data:
        sc.pp.scale(data)
        
    if convert:
        counts = convert_data(data = data, assay = assay, label_column = label_column)
    else:
        counts = data.X
        
    if quantile_normalize:
        quant_norm_data = convert_data(data, assay = assay)
        counts = quantile_normalize(data.X)
    
    # convert string label into numeric label
    labels = data.obs[label_column].unique().tolist()
    labels.sort()
    label = data.obs[label_column].apply(lambda x: labels.index(x))
    
    res = {"matrix": counts, 
           "convert_label": label, 
           "original_label": data.obs[label_column], 
           "sort_uniq_label": labels} 
    
    if save:
        file_name = "Preprocessing_data_{}.pkl".format(record_time())
        with open(file_name, "wb") as output:
            pickle.dump(res, output)
    
    return res

In [122]:
data = pre_processing("./cellhint_demo_folder/cellhint_demo_folder/Spleen.h5ad", assay = ".X", 
                      label_column = "cell_type",
                      convert = True,
                      scale_data = False,
                      save = False)

/home/fengtang/miniconda3/lib/python3.11/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [128]:
def feature_selection(data, 
                      label_column,
                      random_foreast_threshold = None,
                      SVM_threshold = None, 
                      variance_threshold = "zero",
                      mutual_info = True, 
                      chi_square_test = False,
                      F_test = False,
                      method = "all",
                      model  = ["random_foreast", "logistic", "svm"],
                      n_estimators = 100,
                      random_state = 10,
                      kernel = "linear",
                      decision_function_shape = "ovo"):
    """
    A function to do feature seletion based on filtering, embedding and wrapping method respectively or combing those methods together.
    
    Parameters:
    data: A pandas data frame object.
    label_column: The name of cell type column in the data.
    random_foreast_threshold. A float or int value to set the cutoff (feature_importance_) by random foreast model-basedd embedding feature selection.It needs to be specified when model is set in 'random_foreast'.
    SVM_threshold:
    variance_threshold: A string to decide which variance cutoff is used to filter out features."zero" or "median" could be selected. 
    mutual_info: A Bool value decide whether a mutual information method is employed to filtering out features further.
    chi_sqaure_test: A Bool value decide whether a chi square test method is employed to filtering out features further.
    F_test: A Bool value decide whether a F test method is employed to filtering out features further.
    method: A string to decide whether combining filtering, embedding and wrapping feature selection method togther or just using one of them. "all", "filtering", "embedding" and "wrapping" could be selected.
    model: A string to decide which model is used by embedding-based feature selection. "random_foreast", "logistic" and "svm" could be selected.
    n_estimators: The number of trees in the forest.
    random_state:Controls both the randomness of the bootstrapping of the samples used when building trees (if ``bootstrap=True``) and the sampling of thefeatures to consider when looking for the best split at each node.
    kernel: Specifies the kernel type to be used in the support vector machine algorithm.
    decision_function_shape: Whether to return a one-vs-rest ('ovr') decision function of shape (n_samples, n_classes) as all other classifiers, or the originalone-vs-one ('ovo') decision function of libsvm which has shape.(n_samples, n_classes * (n_classes - 1) / 2)
    """
    
    # step 1 - convert category label into numeric label
    print("{} step 1 - converting categoric label into numeric label".format(record_time()))
    le = LabelEncoder().fit(data[label_column])
    label = le.transform(data[label_column])
    data[label_column] = label
        
    X = data.iloc[:, 1:-1]
    y = data.iloc[:, -1]
    
    # step 2 - do feature selection
    print("{} step 2 - do feature selection".format(record_time()))
    print("{} ======== filtering based selection ========".format(record_time()))
    if method == "all":
        
        # filtering-based feature selection
        # filter out by variance 
        if variance_threshold == "zero":
            var_selector = VarianceThreshold()
            X_var = var_selector.fit_transform(X)
            retained_features_by_filter = X.columns[var_selector.get_support()]
            print("* {} {} features remained after filter out features with 0 variance".format(record_time(), X_var.shape[1]))
        
        elif variance_threshold == "median":
            var_selector = VarianceThreshold(np.median(np.var(np.array(X), axis = 0)))
            X_var = var_selector.fit_transform(X)
            retained_features_by_filter = X.columns[var_selector.get_support()]
            print("* {} {} features remained after filter out features below median variance of all features".format(record_time(), X_var.shape[1]))
            
        # filter by chi sqaure test
        if chi_square_test and F_test == False:
            chivalue, pvalues_chi = chi2(X_var, y)
            k = chivalue.shape[0] - (pvalues_chi > 0.05).sum()
            selector = SelectKBest(chi2, k = k)
            X_fschi = selector.fit_transform(X_var, y)
            retained_features_by_filter = retained_features_by_filter[selector.get_support()]
            print("** {} {} features remained after further chi sqaure test filtering".format(record_time(), X_fschi.shape[1]))
            
        # filter by F test
        if F_test and chi_square_test == False:
            F, pvalues_f = f_classif(X_var, y)
            k = F.shape[0] - (pvalues_f > 0.05).sum()
            selector = SelectKBest(f_classif, k = k)
            X_fsF = selector.fit_transform(X_var, y)
            retained_features_by_filter = retained_features_by_filter[selector.get_support()]
            print("** {} {} features remained after further chi sqaure test filtering".format(record_time(), X_fsF.shape[1]))
            
        # filter by mutual infomation
        if (F_test == False and chi_square_test == False) and mutual_info:
            res = mutual_info_classif(X_var, y)
            k = res.shape[0] - sum(res <= 0)
            selector = SelectKBest(mutual_info_classif, k = k)
            X_fsmic = selector.fit_transform(X_var, y)
            retained_features_by_filter = retained_features_by_filter[selector.get_support()]
            print("** {} {} features remained after further mutual information filtering".format(record_time(), X_fsmic.shape[1]))
        
        print("{} ======== embedding based selection ========".format(record_time()))
        # embedding-based on feature selection
        # select by random foreast model
        if model == "random_foreast":
            RFC_ = RandomForestClassifier(n_estimators = 100, random_state = random_state)
            RFC_embedding_selector = SelectFromModel(RFC_, threshold = random_foreast_threshold)
            X_RFC_embedding = RFC_embedding_selector.fit_transform(X, y)
            retained_features_by_embedding = RFC_embedding_selector.get_feature_names_out()
            print("* {} {} features remained after random foreast based embedding filtering".format(record_time(), X_RFC_embedding.shape[1]))
       
        # select by logistic regression model
        elif model == "logistic":
            logistic_ = LogisticRegression(multi_class = "multinomial", random_state = random_state, max_iter=200)
            log_embedding_selector = SelectFromModel(logistic_, norm_order = 1)
            X_log_embedding = log_embedding_selector.fit_transform(X, y)
            retained_features_by_embedding = log_embedding_selector.get_feature_names_out()
            print("* {} {} features remained after logistic regression based embedding filtering".format(record_time(), X_log_embedding.shape[1]))
        
        # select by SVM model
        elif model  == "svm":
            SVC_ = SVC(decision_function_shape = decision_function_shape, kernel = kernel)
            SVC_embedding_selector = SelectFromModel(SVC_, norm_order = 1)
            X_SVC_embedding = SVC_embedding_selector.fit_transform(np.array(X), y)
            retained_features_by_embedding = X.columns[SVC_embedding_selector.get_support()]
            print("* {} {} features remained after svm based embedding filtering".format(record_time(), X_SVC_embedding.shape[1]))
        
        print("{} ======== final feature selection ========".format(record_time()))
        if isinstance(retained_features_by_filter, set) and isinstance(retained_features_by_embedding, set):
            final_feture_selection = retained_features_by_filter.intersection(retained_features_by_embedding)
            print("* {} {} features remained after intersecting the key features found by filtering and embedding-based feature selection method".format(record_time(), len(final_feture_selection)))
        else:
            retained_features_by_filter = set(retained_features_by_filter)
            retained_features_by_embedding = set(retained_features_by_embedding)
            final_feture_selection = retained_features_by_filter.intersection(retained_features_by_embedding)
            print("* {} {} features remained after intersecting the key features found by filtering and embedding-based feature selection method".format(record_time(), len(final_feture_selection)))    
    
    else:
        # filtering-based feature selection
        if method == "filtering" :
            pass
        elif method == "embedding":
            pass
        else:
            pass
    

In [129]:
test = feature_selection(data=data["matrix"].iloc[1:1000, 72870:74370], 
                  label_column="cell_type", 
                  variance_threshold="median",
                  method="all", 
                  model="svm",
                  chi_square_test=True,
                  F_test=False,
                  mutual_info=False
                 )

2023-12-25 20:44:44 step 1 - converting categoric label into numeric label
2023-12-25 20:44:44 step 2 - do feature selection
2023-12-25 20:44:44 ======== filtering based selection ========
* 2023-12-25 20:44:45 747 features remained after filter out features below median variance of all features
** 2023-12-25 20:44:45 413 features remained after further chi sqaure test filtering
2023-12-25 20:44:45 ======== embedding based selection ========
* 2023-12-25 20:44:46 528 features remained after svm based embedding filtering
2023-12-25 20:44:46 ======== final feature selection ========
* 2023-12-25 20:44:46 343 features remained after intersecting the key features found by filtering and embedding-based feature selection method


(999, 1500)